In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

# Hands on 2: Data Preparation

## Handling Missing Values

## (1) Missing Values: Remove them

In [2]:
df.describe().show()

+-------+------------------+------------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|summary|            number|  air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|relative_humidity_3pm|
+-------+------------------+------------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+---------------------+
|  count|              1095|              1092|              1090|                  1091|              1092|                  1092|              1091|                 1089|             1092|                 1095|                 1095|
|   mean|             547.0| 918.8825513138097| 64.933001412

In [3]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [4]:
df.count()

1095

In [5]:
removeAllDF = df.na.drop()

In [6]:
removeAllDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558739|
| stddev|11.168033449415699|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [7]:
removeAllDF.count()

1064

## Missing values : Replace with Mean

In [8]:
from pyspark.sql.functions import avg

# function: fill(double value, scala.collection.Seq<String> cols)
imputeDF = df # create a copy

#go through each column and replace missing values with mean

for x in imputeDF.columns:
    # drop() return df w/o missing values, agg() returns df of aggregate, first() returns first row as Row, [0] to access value 
    meanValue = removeAllDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    # na.fill(), first arg is replacement value, second arg is list of columns
    imputeDF         = imputeDF.na.fill(meanValue, [x])

number 545.0018796992481
air_pressure_9am 918.9031798641055
air_temp_9am 65.02260949558739
avg_wind_direction_9am 142.30675564934032
avg_wind_speed_9am 5.485793050713691
max_wind_direction_9am 148.48042413321312
max_wind_speed_9am 6.9997136588756925
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996038
relative_humidity_9am 34.07743985327712
relative_humidity_3pm 35.14838093290537


In [9]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287075|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219822|
| stddev|11.149948199920226|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [10]:
imputeDF.count()

1095